In [ ]:
# importing libraries

import tensorflow as tf
import numpy as np
import pandas as pd
import glob

In [ ]:
# defining params

# W must be less than or equal to CorrW
W = 20
CorrW = 50 

M = 0
N = 0

# 0-DateTime, 1-Close, 2-Open, 3-High, 4-Low, 5-Vol
SD = 0

# 0-DateTime, 1-Close, 2-Open
MD = 1

In [ ]:
stocks = np.load('prepared_data\\stocks.npy', allow_pickle=True)
markets = np.load('prepared_data\\markets.npy', allow_pickle=True)

print(stocks.shape)
print(markets.shape)

M = markets.shape[0]
N = stocks.shape[0]

In [ ]:
def build_corr_list(stock, market, CorrW):
    market_corr = []
    send = stock.shape[0] 
    mend = market.shape[0]
    while (mend >= CorrW) and (send >= CorrW):
        s_window = stock[send - CorrW : send]
        m_window = market[mend - CorrW : mend]
        corr_matrix = np.corrcoef(s_window, m_window)
        corr = corr_matrix[0, 1]
        market_corr.append(corr)
        send -= 1
        mend -= 1
    return market_corr[::-1]

def trim_2d_list(lst):
    min_length = min([len(x) for x in lst])
    for i in range(len(lst)):
        lst[i] = lst[i][-min_length:]

In [ ]:
all_data = []

for i in range(N):
    # Build a 2D list contatining all the correlations between this stock and all the markets.
    markets_corr = [] 
    for j in range(M):
        corr_list = build_corr_list(stocks[i][:, SD], markets[j][:, MD], CorrW)
        markets_corr.append(corr_list)
    trim_2d_list(markets_corr)

    # Trimming this stock and all the markets
    X = len(markets_corr[0])
    stock_close = stocks[i][-X:, 1]
    stock_open = stocks[i][-X:, 2]
    stock_high = stocks[i][-X:, 3]
    stock_low = stocks[i][-X:, 4]
    stock_vol = stocks[i][-X:, 5]
    trim_markets = []
    for j in range(M):
        trim_markets.append(markets[j][-X:, MD])

    # Concatenate all the previous parts
    data = [stock_close, stock_open, stock_high, stock_low, stock_vol, markets_corr, trim_markets]
    all_data.append(data)

    print('{} of {} done'.format(i + 1, N))
    if (i % 10 == 9) or (i == N - 1):
        np.save('./prepared_data/stocks_correlations_markets_{}'.format((i // 10) + 1), all_data)
        all_data = []
